### Simualting BlackJack

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from blackjack import Game

In [3]:
# usage of the Game class:
# player_num, start_money, bet, deck_num, logging_level, dealer_threshold=17
game = Game(1, 100, 1, 8, "debug")
xlist = list()
game_count = 0

while game.players[0].money>80:
    xlist.append(game.players[0].money)
    if game.cut_card > game.index and game.index < game.deck_length:
        # game.index += 1  # burning the first card.
        game.round += 1
        game.play_round()
    else:
        game = Game(1, game.players[0].money, 1, 8, "debug")
        game.play_round()
    
print(money_list)
plt.plot(money_list)

INFO:Deck: [11  1 13 10  5  2  2  7  5  8  2 11 10  1 12  8  2  5  8  2  8  4  7  7 10
  2 12  4 13  5  7  9  6 11  7 10  5 11  4  8 10  1  6  6  3 12 13  4  6 10
 10  4  9  4 12  1 12  9  5  4  4 12 12  7  6  5  7  7  4  7  9  3 10 11  2
  6  2  1  8 11  4  3  7  8  1 12  4 10 13  3  7  1 11  4 12  9  4  2  1  2
  4  2  2  6  7  7  8 11 10  5 12  6  5 10  9  6  3 10 13  9  1 13  1  3  8
  8 11 10  4  4  8  7  2 12  4  6  9 10  1  9  6  1  6  3  1  5  2  2  2 13
 12  3  7  9  5  2  7  3  2 13  5 10  4 11  1  9  8  4 12 12  3  3 12 11  3
 13 11  9  9  5  4  7 13  8 12  3  3  4  9 10  8  9  8  4  7  5 13 11 11  7
  2  4  9  3  2  8  5  4  6  1 11 11 11 13 10  3 11  6 11  7 12  5 10 13 11
  9 10  1 13  4 11  3  1  5 10  2 13  5  8  7  5 13 11 13  8  7  8 10  2 13
 12 12 10  3 13  4  4  3  2  6  8  3 11 13  9  2  4  1 11 12 10  8  1  9  1
 10  7  3  7 13 10 12 12  7  9 12  1  2  4  7 10  9  2  9  6  5  1  5  6  1
  5 12  1 13  6 10  9 11  1  4 13  3  8  2  3  5  4  3  8  7  8 11 13 11  1
 

INFO:Deck length: 416
INFO:Deck length: 416
INFO:Deck length: 416
INFO:Deck length: 416
INFO:Deck length: 416
INFO:Deck length: 416
INFO:Deck length: 416
INFO:Cut card: 278
INFO:Cut card: 278
INFO:Cut card: 278
INFO:Cut card: 278
INFO:Cut card: 278
INFO:Cut card: 278
INFO:Cut card: 278
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Player-0 hand:[10 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1	Dealer hand:[ 1 10]
DEBUG:Round:1 Lost!	Player hand:[10 10] Dealer hand:[ 1 10] Current bet:1 Earning ratio:-1Earning:-1 Current Money:99
DEBUG:Round:1 Lost!	Player hand:[10 10] Dealer hand:[ 1 10] Current bet:1 Earning ratio:-1Earning:-1 Curr

DEBUG:Round:3	Win!	Player hand:[1 8]	Dealer hand:[10  2  5]	Current bet:1	Earning ratio:1	Earning:1	Current Money:103
DEBUG:Round:3	Win!	Player hand:[1 8]	Dealer hand:[10  2  5]	Current bet:1	Earning ratio:1	Earning:1	Current Money:103
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Player-0 hand:[8 8]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Dealer hand:[2 4]
DEBUG:Round:4	Player splits.	Player hand:[8 8]	Dealer hand:[2 4]
DEBUG:Round:4	Player splits.	Player hand:[8 8]	Dealer hand:[2 4]
DEBUG:Round:4	Player splits.	Player hand:[8 8]	Dealer hand:[2 4]
DEBUG:Round:4	Player splits.	Player hand:[8 8]	Dealer hand:[2 4]
DEBUG:Round:4	Player splits.	Player hand:[

DEBUG:Round:6	Lost!	Player hand:[6 7 5]	Dealer hand:[10 10]	Current bet:1	Earning ratio:-1	Earning:-1	Current Money:102
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Player-0 hand:[10  8]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10  8]	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10  8]	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10  8]	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10  8]	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10  8]	Dealer hand:[ 4 10]
DEBUG:Round:7	Player stands.	Player hand:[10 

DEBUG:Round:10	Push!	Player hand:[10 10]	Dealer hand:[1 9]	Current bet:1	Earning ratio:0	Earning:0	Current Money:99
DEBUG:Round:10	Push!	Player hand:[10 10]	Dealer hand:[1 9]	Current bet:1	Earning ratio:0	Earning:0	Current Money:99
DEBUG:Round:10	Push!	Player hand:[10 10]	Dealer hand:[1 9]	Current bet:1	Earning ratio:0	Earning:0	Current Money:99
DEBUG:Round:10	Push!	Player hand:[10 10]	Dealer hand:[1 9]	Current bet:1	Earning ratio:0	Earning:0	Current Money:99
DEBUG:Round:10	Push!	Player hand:[10 10]	Dealer hand:[1 9]	Current bet:1	Earning ratio:0	Earning:0	Current Money:99
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Player-0 hand:[5 4]
DEBUG:Round:11	Dealer hand:[ 4 10]
DEBUG:Round:11	Dealer hand:[ 4 10]
DEBUG:Round:11	Dealer hand:[ 4 10]
DEBUG:Round:11	Dealer hand:[ 4 10]
DEBUG:Round:11	Dealer hand:[ 4 10]


...

DEBUG:Round:31	Lost!	Player hand:[10 10]	Dealer hand:[ 1 10 10]	Current bet:1	Earning ratio:-1	Earning:-1	Current Money:103.5
DEBUG:Round:31	Lost!	Player hand:[10 10]	Dealer hand:[ 1 10 10]	Current bet:1	Earning ratio:-1	Earning:-1	Current Money:103.5
DEBUG:Round:31	Lost!	Player hand:[10 10]	Dealer hand:[ 1 10 10]	Current bet:1	Earning ratio:-1	Earning:-1	Current Money:103.5
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Player-0 hand:[10  2]
DEBUG:Round:32	Dealer hand:[8 3]
DEBUG:Round:32	Dealer hand:[8 3]
DEBUG:Round:32	Dealer hand:[8 3]
DEBUG

DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Player hits.	Player hand:[ 4  4  3 10 10]	Dealer hand:[ 2 10]
DEBUG:Round:33	Pl

DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Player stands.	Player hand:[1 5 2]	Dealer hand:[7 1]
DEBUG:Round:34	Push!	Player hand:[1 5 2]	Dealer hand:[7 1]	Current bet:1	Earning ratio:0	Earning:0	Current Money:103.5
DEBUG:Round:34	Push!	Player hand:[1 5 2]	Dealer hand:[7 1]	Current bet:1	Earning ratio:0	Earning:0	Current Money:103.5
DEBUG:Round:34	Push!	Player hand:[1 5 2]	Dealer hand:[7 1]	Current bet:1	Earning ratio:0	Earning:0	Current Money:103.5
DEBUG:Round:34	Push!	Player hand:[1 5 2]	Dealer hand:[7 1]	Current bet:1	Earning ratio:0	Earning:0	

DEBUG:Round:36	Player-0 hand:[ 6 10]
DEBUG:Round:36	Player-0 hand:[ 6 10]
DEBUG:Round:36	Player-0 hand:[ 6 10]
DEBUG:Round:36	Player-0 hand:[ 6 10]
DEBUG:Round:36	Player-0 hand:[ 6 10]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Dealer hand:[1 4]
DEBUG:Round:36	Player hits.	Player hand:[ 6 10 10]	Dealer hand:[1 4]
DEBUG:Round:36	Player hits.	Player hand:[ 6 10 10]	Dealer hand:[1 4]
DEBUG:Round:36	Player hits.	Player hand:[ 6 10 10]	Dealer hand:[1 4]
DEBUG:Round:36	Player hits.	Player hand:[ 6 10 10]	Dealer hand:[1 4]
DEBUG:Round:36	Player hits.	Player hand:[ 6 10 10]	Dealer hand:[1 4]
DEBUG:Ro

DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:37	Win!	Player hand:[10  8]	Dealer hand:[ 6 10  6]	Current bet:1	Earning ratio:1	Earning:1	Current Money:104.5
DEBUG:Round:38	P